In [22]:
from pprint import pprint
from paddleocr import PaddleOCR
def ocr2content(img_path):
    # need to run only once to download and load model into memory
    ocr = PaddleOCR(lang='ch')
    content = []
    result = ocr.ocr(img_path, cls=False)
    for idx in range(len(result)):
        res = result[idx]
        for (bbox, (text, prob)) in res:
            content.append((bbox, text))
    return content

content = ocr2content("./whu/image6.png")

[2024/11/27 19:41:27] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/aliancn/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/aliancn/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_tex

In [ ]:
def normalize_bbox(ocr_results):
    """
    将OCR结果的四个顶点坐标归一化为左上和右下角的bbox形式。
    :param ocr_results: OCR结果 [([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], "文本内容")]
    :return: 格式化后的列表 [{"bbox": [x1, y1, x2, y2], "text": "文本内容"}]
    """
    normalized_results = []
    for coords, text in ocr_results:
        x_coords = [point[0] for point in coords]
        y_coords = [point[2] for point in coords]
        bbox = [min(x_coords), min(y_coords), max(x_coords),
                max(y_coords)]  # [左上x, 左上y, 右下x, 右下y]
        normalized_results.append({"bbox": bbox, "text": text})
    return normalized_results

normalize_bbox(content)

[{'bbox': [432.0, 73.0, 637.0, 129.0], 'text': '电子发票'},
 {'bbox': [1144.0, 99.0, 1492.0, 121.0], 'text': '发票号码：24422000000034133596'},
 {'bbox': [727.0, 141.0, 833.0, 158.0], 'text': '国家税务总局'},
 {'bbox': [1140.0, 142.0, 1421.0, 165.0], 'text': '开票日期：2024年04月12日'},
 {'bbox': [720.0, 174.0, 842.0, 210.0], 'text': '湖北省税务局'},
 {'bbox': [59.0, 255.0, 78.0, 279.0], 'text': '购'},
 {'bbox': [101.0, 260.0, 437.0, 288.0], 'text': '名称：黄冈惠鑫资产经营有限公司'},
 {'bbox': [792.0, 255.0, 811.0, 279.0], 'text': '销'},
 {'bbox': [831.0, 260.0, 1074.0, 288.0], 'text': '名称:迈禾科技有限公司'},
 {'bbox': [1520.0, 254.0, 1541.0, 278.0], 'text': '集'},
 {'bbox': [54.0, 273.0, 81.0, 360.0], 'text': '买方信'},
 {'bbox': [789.0, 273.0, 816.0, 359.0], 'text': '售方信'},
 {'bbox': [94.0, 334.0, 743.0, 363.0],
  'text': '统一社会信用代码/纳税人识别号:91421100MA48AEF50M'},
 {'bbox': [826.0, 334.0, 1473.0, 363.0],
  'text': '统一社会信用代码/纳税人识别号:91420111MA4KMELF75'},
 {'bbox': [57.0, 351.0, 78.0, 381.0], 'text': '息'},
 {'bbox': [792.0, 354.0, 811.0, 381.0], '

In [17]:
def remove_noise(normalized_results, keywords=None):
    """
    根据内容过滤噪声字段。
    :param normalized_results: 已归一化的OCR结果 [{"bbox": [...], "text": "文本内容"}]
    :param keywords: 噪声关键词列表，如 ["水印", "背景"]
    :return: 去噪后的字段列表
    """
    if keywords is None:
        keywords = ["水印", "测试", "背景"]

    return [
        item for item in normalized_results
        if item["text"].strip() and all(kw not in item["text"] for kw in keywords)
    ]


In [18]:
def sort_results(normalized_results):
    """
    按坐标排序，先纵向再横向。
    :param normalized_results: 去噪后的OCR结果 [{"bbox": [...], "text": "文本内容"}]
    :return: 排序后的结果
    """
    return sorted(normalized_results, key=lambda x: (x["bbox"][1], x["bbox"][0]))
sort_results_content = sort_results(remove_noise(normalize_bbox(content)))

In [19]:
pprint(sort_results_content)

[{'bbox': [432.0, 73.0, 637.0, 129.0], 'text': '电子发票'},
 {'bbox': [1144.0, 99.0, 1492.0, 121.0], 'text': '发票号码：24422000000034133596'},
 {'bbox': [727.0, 141.0, 833.0, 158.0], 'text': '国家税务总局'},
 {'bbox': [1140.0, 142.0, 1421.0, 165.0], 'text': '开票日期：2024年04月12日'},
 {'bbox': [720.0, 174.0, 842.0, 210.0], 'text': '湖北省税务局'},
 {'bbox': [1520.0, 254.0, 1541.0, 278.0], 'text': '集'},
 {'bbox': [59.0, 255.0, 78.0, 279.0], 'text': '购'},
 {'bbox': [792.0, 255.0, 811.0, 279.0], 'text': '销'},
 {'bbox': [101.0, 260.0, 437.0, 288.0], 'text': '名称：黄冈惠鑫资产经营有限公司'},
 {'bbox': [831.0, 260.0, 1074.0, 288.0], 'text': '名称:迈禾科技有限公司'},
 {'bbox': [54.0, 273.0, 81.0, 360.0], 'text': '买方信'},
 {'bbox': [789.0, 273.0, 816.0, 359.0], 'text': '售方信'},
 {'bbox': [94.0, 334.0, 743.0, 363.0],
  'text': '统一社会信用代码/纳税人识别号:91421100MA48AEF50M'},
 {'bbox': [826.0, 334.0, 1473.0, 363.0],
  'text': '统一社会信用代码/纳税人识别号:91420111MA4KMELF75'},
 {'bbox': [57.0, 351.0, 78.0, 381.0], 'text': '息'},
 {'bbox': [792.0, 354.0, 811.0, 381.0], '

In [20]:
from heapq import merge


def merge_fields(sorted_results, x_threshold=20, y_threshold=10):
    """
    合并相邻字段（同一行或同一逻辑字段）。
    :param sorted_results: 排序后的OCR结果 [{"bbox": [...], "text": "文本内容"}]
    :param x_threshold: 同行字段横向间距阈值
    :param y_threshold: 同行纵向间距阈值
    :return: 合并后的字段列表
    """
    merged_results = []
    current_line = []
    prev_bbox = None

    for item in sorted_results:
        if not current_line:
            current_line.append(item)
            prev_bbox = item["bbox"]
            continue

        # 检查是否属于同一行
        if abs(item["bbox"][1] - prev_bbox[1]) <= y_threshold:
            # 检查是否属于同一字段
            if abs(item["bbox"][0] - prev_bbox[2]) <= x_threshold:
                current_line.append(item)
            else:
                # 当前行结束，合并内容
                merged_text = " ".join([i["text"] for i in current_line])
                merged_bbox = [
                    current_line[0]["bbox"][0],  # 左上角x
                    current_line[0]["bbox"][1],  # 左上角y
                    current_line[-1]["bbox"][2],  # 右下角x
                    current_line[-1]["bbox"][3]  # 右下角y
                ]
                merged_results.append(
                    {"bbox": merged_bbox, "text": merged_text})
                current_line = [item]
        else:
            # 不在同一行，合并当前行内容
            merged_text = " ".join([i["text"] for i in current_line])
            merged_bbox = [
                current_line[0]["bbox"][0],
                current_line[0]["bbox"][1],
                current_line[-1]["bbox"][2],
                current_line[-1]["bbox"][3]
            ]
            merged_results.append({"bbox": merged_bbox, "text": merged_text})
            current_line = [item]
        prev_bbox = item["bbox"]

    # 最后一行处理
    if current_line:
        merged_text = " ".join([i["text"] for i in current_line])
        merged_bbox = [
            current_line[0]["bbox"][0],
            current_line[0]["bbox"][1],
            current_line[-1]["bbox"][2],
            current_line[-1]["bbox"][3]
        ]
        merged_results.append({"bbox": merged_bbox, "text": merged_text})

    return merged_results

merged_content = merge_fields(sort_results_content)

In [21]:
merged_content

[{'bbox': [432.0, 73.0, 637.0, 129.0], 'text': '电子发票'},
 {'bbox': [1144.0, 99.0, 1492.0, 121.0], 'text': '发票号码：24422000000034133596'},
 {'bbox': [727.0, 141.0, 833.0, 158.0], 'text': '国家税务总局'},
 {'bbox': [1140.0, 142.0, 1421.0, 165.0], 'text': '开票日期：2024年04月12日'},
 {'bbox': [720.0, 174.0, 842.0, 210.0], 'text': '湖北省税务局'},
 {'bbox': [1520.0, 254.0, 1541.0, 278.0], 'text': '集'},
 {'bbox': [59.0, 255.0, 78.0, 279.0], 'text': '购'},
 {'bbox': [792.0, 255.0, 811.0, 279.0], 'text': '销'},
 {'bbox': [101.0, 260.0, 437.0, 288.0], 'text': '名称：黄冈惠鑫资产经营有限公司'},
 {'bbox': [831.0, 260.0, 1074.0, 288.0], 'text': '名称:迈禾科技有限公司'},
 {'bbox': [54.0, 273.0, 81.0, 360.0], 'text': '买方信'},
 {'bbox': [789.0, 273.0, 816.0, 359.0], 'text': '售方信'},
 {'bbox': [94.0, 334.0, 743.0, 363.0],
  'text': '统一社会信用代码/纳税人识别号:91421100MA48AEF50M'},
 {'bbox': [826.0, 334.0, 1473.0, 363.0],
  'text': '统一社会信用代码/纳税人识别号:91420111MA4KMELF75'},
 {'bbox': [57.0, 351.0, 78.0, 381.0], 'text': '息'},
 {'bbox': [792.0, 354.0, 811.0, 381.0], '